In [ ]:
import dask.dataframe as dd
from dask.diagnostics import ProgressBar
df = ds.resample({"XTIME": "D"}).max().to_dataframe(
    ).drop(columns = ["XLONG", "XLAT"])
df = df.reset_index()
df["XTIME"] = df["XTIME"].astype(str)
df_2 = ds.to_dataframe()
df_2["hour"] = df_2.index.get_level_values("XTIME").hour.astype(int)
df_2 = df_2.reset_index()
df_2["XTIME"] = df_2["XTIME"].dt.date.astype(str)
df_2 = df_2.set_index(["XTIME", "south_north", "west_east"])
df_i = dd.from_pandas(df.iloc[:132*112*10], chunksize=66)
df_i["hour"] = df_i.apply(lambda x: df_2[df_2["P_mp"]==x["P_mp"]].loc[
    (x["XTIME"], x["south_north"], x["west_east"]), "hour"],
    axis = 1, meta = pd.Series(dtype = str))
with ProgressBar(): df_i.compute()